In [ ]:
# !pip install -Uqq fastbook kaggle waterfallcharts treeinterpreter dtreeviz
# import fastbook
# fastbook.setup_book()

In [ ]:
from fastbook import *
from kaggle import api
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from dtreeviz.trees import *
from IPython.display import Image, display_svg, SVG

pd.options.display.max_rows = 20
pd.options.display.max_columns = 8

In [ ]:
# creds = '{"username":"foo","key":"bar"}'

In [ ]:
# cred_path = Path('~/.kaggle/kaggle.json').expanduser()
# if not cred_path.exists():
#     cred_path.parent.mkdir(exist_ok=True)
#     cred_path.write_text(creds)
#     cred_path.chmod(0o600)

In [ ]:
path = URLs.path('bluebook')
#path

In [ ]:
Path.BASE_PATH = path

In [ ]:
# if not path.exists():
#     path.mkdir()
#     api.competition_download_cli('bluebook-for-bulldozers', path=path)
#     api.competition
#     file_extract(path/'bluebook-for-bulldozers.zip')
    
path.ls(file_type='text')

In [ ]:
df = pd.read_csv(path/'TrainAndValid.csv', low_memory=False)
df.columns

In [ ]:
df['ProductSize'].unique()

In [ ]:
sizes = 'Large','Large / Medium','Medium','Small','Mini','Compact'

In [ ]:
# df['ProductSize'] = df['ProductSize'].astype('category')
# df['ProductSize'].cat.set_categories(sizes, ordered=True, inplace=True)

In [ ]:
dep_var = 'SalePrice'
df[dep_var] = np.log(df[dep_var])

# Decision Trees

In [ ]:
df['saledate'].head()

In [ ]:
df_test = pd.read_csv(path/'Test.csv', low_memory=False)
df_test = add_datepart(df_test, 'saledate')

In [ ]:
df = add_datepart(df, 'saledate')

In [ ]:
' '.join(o for o in df.columns if o.startswith('sale'))

In [ ]:
procs = [Categorify, FillMissing]

In [ ]:
cond = (df.saleYear<2011) | (df.saleMonth<10)

In [ ]:
len(df.saleYear.unique())

In [ ]:
df.saleYear.hist(bins=len(df.saleYear.unique()));

In [ ]:
train_idx = np.where( cond)[0]
valid_idx = np.where(~cond)[0]
splits = (list(train_idx),list(valid_idx))

In [ ]:
valid_idx.size/train_idx.size

In [ ]:
 df = df.astype({'saleWeek' : 'uint32'})

In [ ]:
df['ProductSize'].dtypes

In [ ]:
cont, cat = cont_cat_split(df, 1, dep_var=dep_var)
to = TabularPandas(df, procs, cat, cont, y_names=dep_var, splits=splits)

In [ ]:
# cont, cat

In [ ]:
len(to.train),len(to.valid)

In [ ]:
to.show(3)

In [ ]:
to.items.head(3)

In [ ]:
to.classes['ProductSize']

In [ ]:
xs,y = to.train.xs,to.train.y
valid_xs,valid_y = to.valid.xs,to.valid.y

In [ ]:
m = DecisionTreeRegressor(max_leaf_nodes=4)
m.fit(xs, y);

In [ ]:
draw_tree(m, xs, size=10, leaves_parallel=True, precision=2)

In [ ]:
samp_idx = np.random.permutation(len(y))[:500]
dtreeviz(m, xs.iloc[samp_idx], y.iloc[samp_idx], xs.columns, dep_var,
         fontname='DejaVu Sans', scale=1.6, label_fontsize=10,
         orientation='LR')

In [ ]:
xs.loc[xs['YearMade']<1900, 'YearMade'] = 1950
valid_xs.loc[valid_xs['YearMade']<1900, 'YearMade'] = 1950

In [ ]:
m = DecisionTreeRegressor(max_leaf_nodes=4).fit(xs, y)
dtreeviz(m, xs.iloc[samp_idx], y.iloc[samp_idx], xs.columns, dep_var,
         fontname='DejaVu Sans', scale=1.6, label_fontsize=10,
         orientation='LR')

In [ ]:
m = DecisionTreeRegressor()
m.fit(xs, y);

In [ ]:
y.shape

In [ ]:
def r_mse(pred,y): return round(math.sqrt(((pred-y)**2).mean()), 6)
def m_rmse(m, xs, y): return r_mse(m.predict(xs), y)
m_rmse(m, xs, y)

In [ ]:
m_rmse(m, valid_xs, valid_y)

In [ ]:
m = DecisionTreeRegressor(min_samples_leaf=25)
m.fit(to.train.xs, to.train.y)
m_rmse(m, xs, y), m_rmse(m, valid_xs, valid_y)

In [ ]:
m.get_n_leaves(), len(xs)

In [ ]:
m.get_n_leaves()

# Random Forest

In [ ]:
xs['ProductSize']

In [ ]:
y

In [ ]:
# xs independent vars
# y dependent vars
# n_estimators number of trees
# max_samples rows to sample
# max_features columns to sample
# min_samples_leaf min rows to sample to avoid overfitting
# oob_score out of bag error
# **kwargs keywords argument
# n_jobs=-1 use all cpu cores
def rf(xs, y, n_estimators=40, max_samples=200_000,
       max_features=0.5, min_samples_leaf=4, **kwargs):
    return RandomForestRegressor(n_jobs=-1, n_estimators=n_estimators,
        max_samples=max_samples, max_features=max_features,
        min_samples_leaf=min_samples_leaf, oob_score=True).fit(xs, y)

m = rf(xs, y);

In [ ]:
m_rmse(m, xs, y), m_rmse(m, valid_xs, valid_y)

In [ ]:
preds = np.stack([t.predict(valid_xs) for t in m.estimators_])

In [ ]:
r_mse(preds.mean(0), valid_y)

In [ ]:
plt.plot([r_mse(preds[:i+1].mean(0), valid_y) for i in range(40)]);

In [ ]:
# training set + validation set = development set
# test set

In [ ]:
r_mse(m.oob_prediction_, y)

In [ ]:
preds = np.stack([t.predict(valid_xs) for t in m.estimators_])
preds.shape

In [ ]:
preds_std = preds.std(0)

In [ ]:
preds_std[:5]

In [ ]:
def rf_feat_importance(m, df):
    return pd.DataFrame({'cols':df.columns, 'imp':m.feature_importances_}
                       ).sort_values('imp', ascending=False)

In [ ]:
fi = rf_feat_importance(m, xs)
fi[:10]

In [ ]:
def plot_fi(fi):
    return fi.plot('cols', 'imp', 'barh', figsize=(12,7), legend=False)

plot_fi(fi[:30]);

# Removing Low-Importance Variables

In [ ]:
to_keep = fi[fi.imp>0.005].cols
len(to_keep)

In [ ]:
xs_imp = xs[to_keep]
valid_xs_imp = valid_xs[to_keep]
m = rf(xs_imp, y)

In [ ]:
m_rmse(m, xs_imp, y), m_rmse(m, valid_xs_imp, valid_y)

In [ ]:
len(xs.columns), len(xs_imp.columns)

In [ ]:
plot_fi(rf_feat_importance(m, xs_imp));

# Removing Redundant Features

In [ ]:
cluster_columns(xs_imp)

In [ ]:
def get_oob(df):
    m = RandomForestRegressor(n_estimators=40, min_samples_leaf=15,
        max_samples=50000, max_features=0.5, n_jobs=-1, oob_score=True)
    m.fit(df, y)
    return m.oob_score_

In [ ]:
get_oob(xs_imp)

In [ ]:
# axis = 1 to drop a column
{c:get_oob(xs_imp.drop(c, axis=1)) for c in (
    'saleYear', 'saleElapsed', 'ProductGroupDesc','ProductGroup',
    'fiModelDesc', 'fiBaseModel',
    'Hydraulics_Flow','Grouser_Tracks', 'Coupler_System')}

In [ ]:
# type(xs_imp)

In [ ]:
# xs_imp.drop??

In [ ]:
to_drop = ['saleYear', 'ProductGroupDesc', 'fiBaseModel', 'Grouser_Tracks']
get_oob(xs_imp.drop(to_drop, axis=1))

In [ ]:
xs_final = xs_imp.drop(to_drop, axis=1)
valid_xs_final = valid_xs_imp.drop(to_drop, axis=1)

# (path/'xs_final.pkl').save(xs_final)
# (path/'valid_xs_final.pkl').save(valid_xs_final)

In [ ]:
save_pickle(path/'xs_final.pkl', xs_final)
save_pickle(path/'valid_xs_final.pkl', valid_xs_final)

In [ ]:
# xs_final = (path/'xs_final.pkl').load()
# valid_xs_final = (path/'valid_xs_final.pkl').load()

In [ ]:
xs_final = load_pickle(path/'xs_final.pkl')
valid_xs_final = load_pickle(path/'valid_xs_final.pkl')

In [ ]:
m = rf(xs_final, y)
m_rmse(m, xs_final, y), m_rmse(m, valid_xs_final, valid_y)

# Partial Dependence

In [ ]:
p = valid_xs_final['ProductSize'].value_counts(sort=False).plot.barh()
c = to.classes['ProductSize']
plt.yticks(range(len(c)), c);

In [ ]:
# type(to)

In [ ]:
# to

In [ ]:
to.classes['ProductSize']

In [ ]:
ax = valid_xs_final['YearMade'].hist()

In [ ]:
from sklearn.inspection import plot_partial_dependence

fig,ax = plt.subplots(figsize=(12, 4))
plot_partial_dependence(m, valid_xs_final, ['YearMade','ProductSize'],
                        grid_resolution=20, ax=ax);

In [ ]:
m

In [ ]:
valid_xs_final

# Tree Interpreter

In [ ]:
# !pip install treeinterpreter
# !pip install waterfallcharts

In [ ]:
#hide
import warnings
warnings.simplefilter('ignore', FutureWarning)

from treeinterpreter import treeinterpreter
from waterfall_chart import plot as waterfall

In [ ]:
row = valid_xs_final.iloc[:5]

In [ ]:
row

In [ ]:
prediction,bias,contributions = treeinterpreter.predict(m, row.values)

In [ ]:
prediction[0], bias[0], contributions[0].sum()

In [ ]:
m

In [ ]:
# row.values

In [ ]:
waterfall(valid_xs_final.columns, contributions[0], threshold=0.08,
          rotation_value=45,formatting='{:,.3f}');

In [ ]:
x_lin = torch.linspace(0, 20, steps=40)
y_lin = x_lin + torch.randn_like(x_lin)
plt.scatter(x_lin, y_lin);

In [ ]:
# a different way to make a vector a matrix
# xs_lin = x_lin.unsqueeze(1)
# x_lin.shape,xs_lin.shape

xs_lin = x_lin[:, None]
xs_lin.shape

In [ ]:
m_lin = RandomForestRegressor().fit(xs_lin[:30], y_lin[:30])

In [ ]:
plt.scatter(x_lin, y_lin, 20)
plt.scatter(x_lin, m_lin.predict(xs_lin), color='red', alpha=0.5)

In [ ]:
df_dom = pd.concat([xs_final, valid_xs_final])
is_valid = np.array([0]*len(xs_final) + [1]*len(valid_xs_final))

m = rf(df_dom, is_valid)
rf_feat_importance(m, df_dom)[:6]

In [ ]:
m = rf(xs_final, y)
print('orig', m_rmse(m, valid_xs_final, valid_y))

for c in ('SalesID','saleElapsed','MachineID'):
    m = rf(xs_final.drop(c,axis=1), y)
    print(c, m_rmse(m, valid_xs_final.drop(c,axis=1), valid_y))

In [ ]:
time_vars = ['SalesID','MachineID']
xs_final_time = xs_final.drop(time_vars, axis=1)
valid_xs_time = valid_xs_final.drop(time_vars, axis=1)

m = rf(xs_final_time, y)
m_rmse(m, valid_xs_time, valid_y)

In [ ]:
xs['saleYear'].hist();

In [ ]:
filt = xs['saleYear']>2004
xs_filt = xs_final_time[filt]
y_filt = y[filt]

m = rf(xs_filt, y_filt)
m_rmse(m, xs_filt, y_filt), m_rmse(m, valid_xs_time, valid_y)

# Using Neural Network

In [ ]:
df_nn = pd.read_csv(path/'TrainAndValid.csv', low_memory=False)

In [ ]:
df_nn['ProductSize']

In [ ]:
df_nn['ProductSize'] = df_nn['ProductSize'].astype('category')

In [ ]:
df_nn['ProductSize']

In [ ]:
sizes

In [ ]:
df_nn['ProductSize'].cat.set_categories(sizes, ordered=True, inplace=True)

In [ ]:
df_nn['ProductSize']

In [ ]:
# df_nn[dep_var] = np.log(df_nn[dep_var])

In [ ]:
df_nn[dep_var]

In [ ]:
df_nn[dep_var] = np.log(df_nn[dep_var])
df_nn[dep_var]

In [ ]:
# np.log??

In [ ]:
df_nn

In [ ]:
df_nn = add_datepart(df_nn, 'saledate')

In [ ]:
df_nn

In [ ]:
xs_final_time.columns

In [ ]:
df_nn_final = df_nn[list(xs_final_time.columns) + [dep_var]]

In [ ]:
cont_nn,cat_nn = cont_cat_split(df_nn_final, max_card=9000, dep_var=dep_var)

In [ ]:
cont_nn

In [ ]:
cat_nn

In [ ]:
cont_nn.append('saleElapsed')

In [ ]:
cont_nn

In [ ]:
cat_nn.remove('saleElapsed')

In [ ]:
df_nn_final.dtypes

In [ ]:
# df_nn_final['saleElapsed'] = df_nn_final['saleElapsed'].astype(int)
df_nn_final.astype({'saleElapsed': 'int64'}).dtypes

In [ ]:
df_nn_final.dtypes

In [ ]:
cat_nn

In [ ]:
df_nn_final[cat_nn].nunique()

In [ ]:
xs_filt2 = xs_filt.drop('fiModelDescriptor', axis=1)

In [ ]:
type(xs_filt)

In [ ]:
xs_filt2.columns

In [ ]:
valid_xs_time2 = valid_xs_time.drop('fiModelDescriptor', axis=1)

In [ ]:
valid_xs_time2

In [ ]:
m2 = rf(xs_filt2, y_filt)

In [ ]:
m2

In [ ]:
y_filt

In [ ]:
m_rmse(m2, xs_filt2, y_filt), m_rmse(m2, valid_xs_time2, valid_y)

In [ ]:
m_rmse(m, xs_filt, y_filt), m_rmse(m, valid_xs_time, valid_y)

In [ ]:
cat_nn.remove('fiModelDescriptor')

In [ ]:
cat_nn

In [ ]:
procs

In [ ]:
procs_nn = [Categorify, FillMissing, Normalize]

In [ ]:
procs_nn

In [ ]:
# to_nn = TabularPandas(df_nn_final, procs_nn, cat_nn, cont_nn,
#                      splits=splits, y_names=dep_var)

In [ ]:
df_nn_final.dtypes

In [ ]:
df_nn_final.astype({'saleElapsed': 'int64'}).dtypes

In [ ]:
# df_nn_final['saleElapsed'] = df_nn_final['saleElapsed'].astype(int)

In [ ]:
to_nn = TabularPandas(df_nn_final, procs_nn, cat_nn, cont_nn,
                      splits=splits, y_names=dep_var)

In [ ]:
dls = to_nn.dataloaders(1024)

In [ ]:
y = to_nn.train.y
y.min(0), y.max()

In [ ]:
from fastai.tabular.all import *

In [ ]:
learn = tabular_learner(dls, y_range=(8, 12), layers=[500, 250],
                       n_out=1, loss_func=F.mse_loss)

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(5, 1e-2)

In [ ]:
preds, targs = learn.get_preds()

In [ ]:
r_mse(preds, targs)

In [ ]:
learn.save('nn')

# Ensembling

In [ ]:
rf_preds = m.predict(valid_xs_time)

In [ ]:
rf_preds = m.predict(valid_xs_time)
ens_preds = (to_np(preds.squeeze()) + rf_preds) /2

In [ ]:
r_mse(ens_preds,valid_y)